In [6]:
import mlflow
import openai
from dotenv import load_dotenv
import os
load_dotenv()

True

In [7]:
mlflow.openai.autolog()

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("OpenAI")

api_key = os.environ.get("OPENAI_API_KEY")
openai_client = openai.OpenAI(api_key=api_key)

messages = [
    {
        "role": "user",
        "content": "What is the capital of France?",
    }
]

response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0.1,
    max_tokens=100,
)

Trace(request_id=edf8a9612bd0471a853aefb4c9f3b471)

In [9]:
mlflow.openai.autolog()
client = openai.OpenAI(api_key=api_key)

stream = client.chat.completions.create(
    model = 'gpt-4o-mini-2024-07-18',
    messages =[
        {'role':'user',
         'content':' do you know about mukhammad fahlevi ali rafsanjani?'}
    ],
    stream=True,
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

As of my last knowledge update in October 2021, I don't have any specific information about an individual named Mukhammad Fahlevi Ali Rafsanjani. If he has come into prominence or relevance after that date, I may not be aware of updates or developments regarding him. If you have specific details or context about him, I may be able to help you better!

Trace(request_id=a078f503c3a344709def108b64260960)

In [ ]:
import openai

# Enable trace logging
mlflow.openai.autolog()

client = openai.AsyncOpenAI(api_key=api_key)

response = await client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=[
        {"role": "user", "content": "How fast would a glass of water freeze on Titan?"}
    ],
    # Async streaming is also supported
    # stream=True

)

In [ ]:
import openai
from mlflow.entities import SpanType
# Enable trace logging
mlflow.openai.autolog()

@mlflow.trace(span_type=SpanType.TOOL)
def get_weather(city:str) -> str:
    if city == "tokyo":
        return "sunny"
    elif city == "Paris":
        return 'rainy'
    return 'unknown'

tools = [
    {
        "type":'function',
        "function":{
            "name":"get_weather",
            "parameter":{
                'type':"object",
                'properties':{"city":{'type':'string'}},
            },
        },
    }
]

_tool_functions = {'get_weather':get_weather}

@mlflow.trace(span_type=SpanType.AGENT)
def run_tool_agent(question:str):
    messages=[{"role":"user","content":question}]
client = openai.AsyncOpenAI(api_key=api_key)

response = await client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messages,
    tools=tools
    # Async streaming is also supported
    # stream=True

)
ai_msg = response.choices[0].message
    messages.append(ai_msg)

    # If the model request tool call(s), invoke the function with the specified arguments
    if tool_calls := ai_msg.tool_calls:
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            if tool_func := _tool_functions.get(function_name):
                args = json.loads(tool_call.function.arguments)
                tool_result = tool_func(**args)
            else:
                raise RuntimeError("An invalid tool is returned from the assistant!")

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": tool_result,
                }
            )

        # Sent the tool results to the model and get a new response
        response = client.chat.completions.create(
            model="gpt-4o-mini", messages=messages
        )

    return response.choices[0].message.content


# Run the tool calling agent
question = "What's the weather like in Paris today?"
answer = run_tool_agent(question)